In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("raw_candidate_data.csv")
df


,candidate_id,name,email,phone,submission_date,score
0,101,Rohan Sharma,rohan@gmail.com,9.876543e+09,10-02-2026,85
1,102,Anjali Mehta,anjali@gmail.com,9.876543e+09,11/02/2026,90
2,103,NaN,rahul@gmail.com,1.000000e+10,12-02-2026,88
3,104,Priya Singh,PRIYA@gmail.com,NaN,13/02/2026,91
4,105,Rohan Sharma,rohan@gmail.com,9.876543e+09,10-02-2026,85
5,106,Arjun Verma,arjun@gmail.com,8.888889e+09,NaN,92


In [2]:
df['name'] = df['name'].str.strip()


In [3]:
df['email'] = df['email'].str.lower()


In [4]:
df['name'] = df['name'].fillna("Unknown")
df['phone'] = df['phone'].fillna("Not Provided")
df['submission_date'] = df['submission_date'].fillna("01-01-2026")

In [5]:
df['submission_date'] = pd.to_datetime(df['submission_date'], errors='coerce')

In [6]:
df = df.drop_duplicates()

In [7]:
df['phone_valid'] = df['phone'].apply(lambda x: True if len(str(x)) == 10 else False)

In [8]:
print("Missing Values:\n", df.isnull().sum())
print("\nTotal Records:", len(df))

Missing Values:
 candidate_id       0
name               0
email              0
phone              0
submission_date    2
score              0
phone_valid        0
dtype: int64

Total Records: 6


In [9]:
df.to_csv("cleaned_candidate_data.csv", index=False)

In [10]:
import sqlite3

In [11]:
conn = sqlite3.connect("cleaned_data_database.db")

In [12]:
df.to_sql("cleaned_candidates", conn, if_exists="replace", index=False)

6

In [14]:
#Total Records
query1 = """
SELECT COUNT(*) AS total_records
FROM cleaned_candidates;
"""

pd.read_sql_query(query1, conn)


,total_records
0,6


In [16]:
#Check Missing Names 
query2 = """
SELECT COUNT(*) AS missing_names
FROM cleaned_candidates
WHERE name = 'Unknown';
"""

pd.read_sql_query(query2, conn)

,missing_names
0,1


In [17]:
#Find Invalid Phone Numbers
query3 = """
SELECT candidate_id, phone
FROM cleaned_candidates
WHERE phone_valid = 0;
"""

pd.read_sql_query(query3, conn)

,candidate_id,phone
0,101,9876543210.0
1,102,9876543210.0
2,103,9999999999.0
3,104,Not Provided
4,105,9876543210.0
5,106,8888888888.0


In [19]:
#Average Score (Aggregation)
query4 = """
SELECT AVG(score) AS average_score
FROM cleaned_candidates;
"""

pd.read_sql_query(query4, conn)


,average_score
0,88.5


In [21]:
#Records Submitted After Specific Date (Filter)
query5 = """
SELECT candidate_id, submission_date
FROM cleaned_candidates
WHERE submission_date > '2026-02-11';
"""

pd.read_sql_query(query5, conn)

,candidate_id,submission_date
0,101,2026-10-02 00:00:00
1,103,2026-12-02 00:00:00
2,105,2026-10-02 00:00:00


In [22]:
conn.close()